#### 1.baseline_model_with_26_predictors

In [ ]:
#import
import random
import polars as pl
import pandas as pd
import numpy as np
from sklearn import model_selection
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split

In [ ]:
# Data
df_final = pd.read_csv('df-updated-msc-group.csv')
df_final = df_final.drop(['SEQN','Unnamed: 0'],axis=1)
df_final

,dsst_score,cerad_score,af_score,age,sex,education,marital_status,household_income,diabetes,phq,...,avg_mvp,std_mvp,avg_mims,std_mims,max_mims,min_mims,avg_slp_onset,std_slp_onset,avg_slp_wakeup,std_slp_wakeup
0,33,21,18,80,0,5.0,1.0,10.0,0.0,0,...,47.857143,14.690457,6.204291,7.083150,40.715,0.0,14.620833,11.884783,8.706250,1.679520
1,38,16,12,80,0,3.0,2.0,1.0,0.0,4,...,24.142857,9.007933,5.219149,6.733183,35.272,0.0,21.335417,0.438471,7.331250,1.929953
2,26,21,17,70,0,3.0,3.0,11.0,0.0,0,...,17.714286,10.403754,4.979287,6.628381,47.541,0.0,22.306250,0.848432,8.652083,1.402476
3,47,28,15,65,1,4.0,1.0,NaN,1.0,3,...,202.857143,72.508456,11.875532,11.619941,61.251,0.0,10.272917,10.369870,7.725000,0.733982
4,25,21,9,77,1,1.0,5.0,4.0,1.0,4,...,121.857143,23.828754,9.597925,10.521175,63.693,0.0,21.812500,0.971774,6.281250,1.291623
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2436,21,34,16,80,1,4.0,2.0,7.0,0.0,2,...,33.428571,19.560650,4.979868,7.909078,49.962,0.0,18.516667,7.468272,10.018750,0.529858
2437,43,28,15,64,0,3.0,3.0,5.0,1.0,5,...,22.571429,12.421180,5.263271,6.875107,53.081,0.0,18.418750,0.910845,9.733333,0.807160
2438,55,34,16,61,0,2.0,1.0,9.0,0.0,3,...,680.666667,700.557159,12.333695,11.021239,61.870,0.0,8.137500,10.366420,9.450000,1.117123
2439,44,15,8,61,0,2.0,1.0,10.0,1.0,0,...,109.285714,53.760935,8.347976,9.104489,69.347,0.0,21.314583,3.301123,5.841667,1.965738


In [ ]:
# Get names of features and targets
X = df_final.drop(['cerad_score', 'af_score', 'dsst_score'],axis=1)
y = df_final[['cerad_score', 'af_score', 'dsst_score']]

Split the entire data set into a training and testing set

In [ ]:
# Split
xtrain,xvalid,ytrain,yvalid=train_test_split(X,y,test_size=0.2,random_state=0)

# Initialize the imputer
imputer = KNNImputer(n_neighbors=1)
xtrain = imputer.fit_transform(xtrain)
xvalid = imputer.transform(xvalid)

xtrain = pd.DataFrame(xtrain)
xvalid = pd.DataFrame(xvalid)
ytrain = pd.DataFrame(ytrain)
yvalid = pd.DataFrame(yvalid)

xtrain.columns = X.columns
xvalid.columns = X.columns

In [ ]:
# Save
xtrain.to_csv(f'xtrain.csv',index=0)
xvalid.to_csv(f'xvalid.csv',index=0)

ytrain.to_csv(f'ytrain.csv',index=0)
yvalid.to_csv(f'yvalid.csv',index=0)

In [ ]:
# Imports
import statistics as stats
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklearn.linear_model import LinearRegression

In [ ]:
rsq_cerad, rsq_af, rsq_dsst = [], [], [] # Lists to store R^2 values
mae_cerad, mae_af, mae_dsst = [], [], [] # Lists to store MAE values
mse_cerad, mse_af, mse_dsst = [], [], [] # Lists to store MSE values

In [ ]:
ytrain_cerad = ytrain[['cerad_score']]
ytrain_af = ytrain[['af_score']]
ytrain_dsst = ytrain[['dsst_score']]

yvalid_cerad = yvalid[['cerad_score']]
yvalid_af = yvalid[['af_score']]
yvalid_dsst = yvalid[['dsst_score']]

# Initialize and fit the models
model_cerad = LinearRegression().fit(xtrain, ytrain_cerad)
model_af = LinearRegression().fit(xtrain, ytrain_af)
model_dsst = LinearRegression().fit(xtrain, ytrain_dsst)

# Predict and append
preds_cerad = model_cerad.predict(xvalid)
preds_af = model_af.predict(xvalid)
preds_dsst = model_dsst.predict(xvalid)

# Print R^2 and mean scores
print(f'Mean R^2 for the CERAD model: {r2_score(yvalid_cerad, preds_cerad)}')
print(f'Mean R^2 for the DSST model: {r2_score(yvalid_dsst, preds_dsst)}')
print(f'Mean R^2 for the AF model: {r2_score(yvalid_af, preds_af)}')
print(f"\n")
print(f'Mean MAE for the CERAD model: {mean_absolute_error(yvalid_cerad, preds_cerad)}')
print(f'Mean MAE for the DSST model: {mean_absolute_error(yvalid_dsst, preds_dsst)}')
print(f'Mean MAE for the AF model: {mean_absolute_error(yvalid_af, preds_af)}')
print(f"\n")
print(f'Mean MSE for the CERAD model: {mean_squared_error(yvalid_cerad, preds_cerad)}')
print(f'Mean MSE for the DSST model: {mean_squared_error(yvalid_dsst, preds_dsst)}')
print(f'Mean MSE for the AF model: {mean_squared_error(yvalid_af, preds_af)}')

Mean R^2 for the CERAD model: 0.2137977488836007
Mean R^2 for the DSST model: 0.4410561213265467
Mean R^2 for the AF model: 0.14529719218913606


Mean MAE for the CERAD model: 4.51207349259914
Mean MAE for the DSST model: 10.30992069446619
Mean MAE for the AF model: 4.05945719649191


Mean MSE for the CERAD model: 32.28167400060314
Mean MSE for the DSST model: 168.25692952952437
Mean MSE for the AF model: 25.254460793015163
